In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard
from keras import optimizers

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

import pickle
import time

#tf.compat.v1.disable_eager_execution()

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=90,
        validation_split=0.15)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/training data/images2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/training data/images2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

'''test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-classification/images/test data",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)'''

NAME = "defect-classification-CNN-64*2-{}".format(int(time.time()))


model = load_model("defect-detection_model.h5")
model.load_weights ("defect-detection_weights.h5")

model.summary()


/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

Found 17850 images belonging to 2 classes.
Found 3150 images belonging to 2 classes.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
conv2

In [2]:
test_datagen2 = ImageDataGenerator(rescale=1./255)

test_generator2 = test_datagen2.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/test data_2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)

Found 222 images belonging to 1 classes.


In [3]:
STEP_SIZE_TEST2=test_generator2.n//test_generator2.batch_size

In [4]:

test_generator2.reset()
pred=model.predict_generator(test_generator2,
steps=STEP_SIZE_TEST2,
verbose=1)

222/222 [==============================] - 12s 52ms/step


In [5]:
import numpy as np
import pandas as pd

predicted_class_indices=np.argmax(pred,axis=1)

cl = np.round(pred)


In [9]:
idx_to_cls = {v: k for k, v in valid_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(test_generator2.filenames))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})

results.to_csv("results2.csv",index=False)

In [45]:
#for only defected

idx_to_cls = {v: k for k, v in valid_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(test_generator2.filenames))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})
results = results[results.class_name != 'nodefect']
results.to_csv("results_defect.csv",index=False)


In [52]:
valid_generator2 = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/training data/images2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42,
    subset="validation"
)


Found 3150 images belonging to 2 classes.


In [53]:
STEP_SIZE_VALID2=valid_generator2.n//valid_generator2.batch_size

In [55]:
valid_generator2.reset()
pred=model.predict_generator(valid_generator2,
steps=STEP_SIZE_VALID2,
verbose=1)

3150/3150 [==============================] - 79s 25ms/step


In [56]:
import numpy as np
import pandas as pd

predicted_class_indices=np.argmax(pred,axis=1)

cl = np.round(pred)

In [57]:
#valid data

idx_to_cls = {v: k for k, v in valid_generator2.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(valid_generator2.filenames))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})

results.to_csv("results_valid.csv",index=False)

In [58]:
#valid data only defected

idx_to_cls = {v: k for k, v in valid_generator2.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(valid_generator2.filenames))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})
results = results[results.class_name != 'nodefect']
results.to_csv("results_valid_defected.csv",index=False)